### Taxonomic Profiling on MAGS
- Can't figure out on anvio so for now trying other options
- option 1: sylph 
- option 2: kraken

##### Sylph
https://github.com/bluenote-1577/sylph
- average nucleic identity (ANI) querying or taxonomic profiling

In [ ]:
#activate unity interactive job..need high memory for installation
salloc -p cpu --mem 150G

In [ ]:
%conda init --all

In [ ]:
#module load miniconda/22.11.1-1
#conda create -n taxa
%conda activate taxa

In [ ]:
#install sylph package
conda install -c bioconda sylph
#if above doesnt work do: 
conda install -c bioconda -c conda-forge sylph

In [ ]:
#set parameters
SAMPLENAME=mcav
CONTIGPATH=/project/pi_sarah_gignouxwolfsohn_uml_edu/brooke/results/mcav_assembly3
CONTIGFILE=mcav.contigs-fixed.fa

In [ ]:
#sketch paired-end reads
sylph sketch -1 ./trimmed/*_R1_001_val_1.fq -2 ./trimmed/*_R2_001_val_2.fq -d "$SAMPLENAME"_sketches -t 50 
#decrease -c (default =200)...

In [ ]:
#create database if contigs are all in one fasta file
sylph sketch $CONTIGPATH/$CONTIGFILE -i -o ./working/"$SAMPLENAME"_contig_database

In [ ]:
# query for ANI 
sylph query "$SAMPLENAME"_sketches/*.sylsp ./working/"$SAMPLENAME"_contig_database.syldb
#output is statistics for each genome against our reads
#redirect output??
#Thus sylph query is not a profiler. It does not tell you the abundance of the genomes in your sample, just how similar your query genome is to the genomes within the sample.

In [ ]:
#Import taxa database: https://github.com/bluenote-1577/sylph/wiki/Pre%E2%80%90built-databases
wget https://storage.googleapis.com/sylph-stuff/v0.3-c200-gtdb-r214.syldb -O gtdb_database.syldb

In [ ]:
#move database into folder called gtdb_genomes_reps_r214
mkdir working/gtdb_genomes_reps_r214
mv gtdb_database.syldb working/gtdb_genomes_reps_r214/gtdb_database.syldb

#set path to GTDB
DB=working/gtdb_genomes_reps_r214/gtdb_database.syldb
#set input & output path
INDIR=mcav_sketches
OUTDIR=results/taxonomy

In [ ]:
#maps samples against contig database???
#read more: https://github.com/bluenote-1577/sylph/wiki/sylph-cookbook#creating-a-database-of-fasta-files
sylph profile "$SAMPLENAME"_sketches/*.sylsp ./working/"$SAMPLENAME"_contig_database.syldb -t 50 -o ./results/"$SAMPLENAME"_sylph_profile.tsv

In [ ]:
#map reads against GTDB 
sylph profile $INDIR/*_MCAV_R1_001_val_1.fq.paired.sylsp "$DB" -t 10 -o $OUTDIR/sylph_GTDB.tsv

In [ ]:
#get taxonomic profile 
#clone repo (in "working" dir)
git clone https://github.com/bluenote-1577/sylph
python working/sylph/scripts/sylph_to_taxprof.py -s $OUTDIR/sylph_GTDB.tsv -m  working/sylph/scripts/gtdb_r214_metadata.tsv.gz -o $OUTDIR/prefix_.tsv

#results: only 6 samples have hits on the reference genomes database

In [ ]:
#convert to tsv

# Loop over all files 
for input_file in prefix_*_R1_001_val_1.fq.sylphmpa; do
    # Extract the unique identifier from the file name
    unique_identifier=$(echo "$input_file" | sed 's/^prefix_\(.*\)_R1_001_val_1\.fq\.sylphmpa$/\1/')

    # Generate the output TSV file name
    output_file="${unique_identifier}.tsv"

    # Process the input file and save the result to the output file
    awk -F'\t' '{for (i=1; i<=NF; i++) printf "%s%s", $i, (i<NF ? "\t" : "\n")}' "$input_file" > "$output_file"

    echo "Processed $input_file and saved as $output_file"
done


In [ ]:
#repeating with OceanDNA database
wget https://storage.googleapis.com/sylph-stuff/OceanDNA-c200-v0.3.syldb
mv OceanDNA-c200-v0.3.syldb working/gtdb_genomes_reps_r214/OceanDNA-c200-v0.3.syldb
oceanDB=working/gtdb_genomes_reps_r214/OceanDNA-c200-v0.3.syldb

sylph profile $INDIR/*_MCAV_R1_001_val_1.fq.paired.sylsp "$oceanDB" -t 10 -o $OUTDIR/sylph_ocean.tsv

##### Kraken
- https://ccb.jhu.edu/software/kraken/MANUAL.html

In [ ]:
module load kraken2/2.1.2

In [ ]:
# download SILVA database
wget -c url https://genome-idx.s3.amazonaws.com/kraken/16S_Silva132_20200326.tgz
#more db options (thinking about 'standard': https://benlangmead.github.io/aws-indexes/k2
# unzip 
tar -xvzf //project/pi_sarah_gignouxwolfsohn_uml_edu/Reference_genomes/ref_databases/16S_Silva132_20200326.tgz

In [ ]:
# set DB path
DBNAME=//project/pi_sarah_gignouxwolfsohn_uml_edu/Reference_genomes/ref_databases/16S_SILVA132_k2db
# build kraken database 
kraken2-build --db $DBNAME --special silva --threads 24

# classify paired-end seqs 
# kraken2 --paired --classified-out cseqs'#'.fq seqs_1.fq seqs_2.fq --output 
# example:
# kraken2 --db 16S_SILVA132_k2db 
        --threads 4                 #number of threads
        --report SAMPLE.kreport2    #kraken-style report (REQUIRED FOR BRACKEN)
        --paired SAMPLE_1.fq SAMPLE_2.fq > SAMPLE.kraken2

# not being thorough ... going to use assmebled contigs and come back to this if needed
# https://www.biostars.org/p/402619/
SAMPLENAME=mcav
CONTIGPATH=/project/pi_sarah_gignouxwolfsohn_uml_edu/brooke/working/mcav_assembly_redo
seqs=$SAMPLENAME.contigs.fa
OUTDIR=//project/pi_sarah_gignouxwolfsohn_uml_edu/brooke/working/mcav_assembly_redo/taxonomy/kraken
mkdir $OUTDIR

kraken2 --db $DBNAME $CONTIGPATH/$seqs --threads 24 

In [ ]:
#!/bin/bash
#SBATCH -c 24  # Number of Cores per Task
#SBATCH --mem=180G  # Requested Memory
#SBATCH -p cpu  # Partition
#SBATCH -t 24:00:00  # Job time limit
#SBATCH --mail-type=ALL
#SBATCH -o //project/pi_sarah_gignouxwolfsohn_uml_edu/brooke/bash_scripts/slurm-kraken-%j.out  # %j = job ID


module load kraken2/2.1.2

# i think this specifically downloads NCBI and RefSeq
DBNAME=//project/pi_sarah_gignouxwolfsohn_uml_edu/Reference_genomes/ref_databases/standard
#kraken2-build --standard --threads 24 --db $DBNAME

READSPATH=/project/pi_sarah_gignouxwolfsohn_uml_edu/brooke/trimmed
OUTDIR=//project/pi_sarah_gignouxwolfsohn_uml_edu/brooke/working/mcav_assembly_redo/taxonomy/kraken

for f in T1_12_2022 T1_13_2022 T1_16_2019 T1_20_2019 T1_24_2019 T1_40_2022 T1_57_2022 T1_70_2022 T2_10_2022 T2_16_2019 T3_13_2022 T3_14_2019 T3_15_2019 T3_19_2022 T3_1_2019 T3_40_2022 T3_48_2022 T3_49_2022 T3_51_2022 T3_60_2022 T3_8_2019 T3_9_2019
do
kraken2 --db $DBNAME \
        --threads 24             \    #number of threads
        --report $OUTDIR/$f.kreport2 \   #kraken-style report (REQUIRED FOR BRACKEN)
        --paired $f_MCAV_R1_001_val_1.fq $f_MCAV_R2_001_val_2.fq > $OUTDIR/$f.kraken2
done

#bash script: kraken
#JOB ID: 15982456